In [1]:
# Importing Libraries
import sqlite3
import pandas as pd
import plotly.express as px
import source_to_landing
import landing_to_sql

## Functions for querying and filtering stored data

In [2]:
# Query and Filter Functions
def query_db(sql_query:str) -> pd.DataFrame():
    """
    This function allows an easy implementation with SQL query into the historical flights database. Returning DataFrame with the Query's result.
    
    Args:
        sql_query(str): SQL Query.
        
    Returns:
        df(pd.DataFrame): pandas.DataFrame containing the query results.
    """
    connection = sqlite3.connect("database/historical_flights.db")
    df = pd.read_sql_query(sql_query, connection)
    return df


def filter_db(keyword:str ,option:str = "airline") -> pd.DataFrame():
    """
    This function filter the db according to the option selected and returns a DataFrame with the results.
    
    Args:
        keyword(str): Keyword used in filter.
        option(str): Filter option selected. Default value:"airline".
            Examples: "airline", "origin", "destination", "aircraft"
        
    Returns:
        df(pd.DataFrame): pandas.DataFrame containing the query results.
    """
    connection = sqlite3.connect("database/historical_flights.db")
    
    if option == "airline":
        query = f"SELECT * FROM flights WHERE airline_name = '{keyword}'"
        
    elif option == "origin":
        query = f"SELECT * FROM flights WHERE depart_airport = '{keyword}'"
        
    elif option == "destination":
        query = f"SELECT * FROM flights WHERE arrival_airport = '{keyword}'"
        
    elif option == "aircraft":
        query = f"SELECT * FROM flights WHERE aircraft_registration = '{keyword}'"
    
    df = pd.read_sql_query(query, connection)

    return df

### Examples using filtering function. *(The query function already is going to be used on the next sections)*

In [160]:
df = filter_db("Schiphol","origin")

df

,id,flight_date,flight_status,flight_number,flight_iata,flight_icao,airline_name,airline_iata,airline_icao,depart_airport,...,arrival_delay,arrival_scheduled,arrival_estimated,arrival_actual,arrival_estimated_runway,arrival_actual_runway,aircraft_registration,aircraft_iata,aircraft_icao,aircraft_icao24
0,2024-04-16DL9388None,2024-04-16,scheduled,9388,DL9388,DAL9388,Delta Air Lines,DL,DAL,Schiphol,...,41,2024-04-16T10:40:00+00:00,2024-04-16T10:40:00+00:00,,,,None,None,None,None
1,2024-04-16DL9476None,2024-04-16,cancelled,9476,DL9476,DAL9476,Delta Air Lines,DL,DAL,Schiphol,...,,2024-04-16T10:55:00+00:00,2024-04-16T10:55:00+00:00,,,,None,None,None,None
2,2024-04-16DL9345None,2024-04-16,cancelled,9345,DL9345,DAL9345,Delta Air Lines,DL,DAL,Schiphol,...,,2024-04-16T09:30:00+00:00,2024-04-16T09:30:00+00:00,,,,None,None,None,None
3,2024-04-16DL9595None,2024-04-16,landed,9595,DL9595,DAL9595,Delta Air Lines,DL,DAL,Schiphol,...,9,2024-04-16T10:45:00+00:00,2024-04-16T10:45:00+00:00,2024-04-16T10:53:00+00:00,2024-04-16T10:53:00+00:00,2024-04-16T10:53:00+00:00,None,None,None,None
4,2024-04-16DL9624None,2024-04-16,scheduled,9624,DL9624,DAL9624,Delta Air Lines,DL,DAL,Schiphol,...,67,2024-04-16T11:05:00+00:00,2024-04-16T11:05:00+00:00,,,,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1488,2024-04-20DL133None,2024-04-20,landed,133,DL133,DAL133,Delta Air Lines,DL,DAL,Schiphol,...,15,2024-04-20T10:40:00+00:00,2024-04-20T10:40:00+00:00,2024-04-20T10:55:00+00:00,2024-04-20T10:55:00+00:00,2024-04-20T10:55:00+00:00,None,None,None,None
1489,2024-04-20DL71A333,2024-04-20,landed,71,DL71,DAL71,Delta Air Lines,DL,DAL,Schiphol,...,2,2024-04-20T12:10:00+00:00,2024-04-20T12:10:00+00:00,2024-04-20T12:05:00+00:00,2024-04-20T12:05:00+00:00,2024-04-20T12:05:00+00:00,N820NW,A333,A333,AB321C
1490,2024-04-20DL143None,2024-04-20,scheduled,143,DL143,DAL143,Delta Air Lines,DL,DAL,Schiphol,...,147,2024-04-20T10:15:00+00:00,2024-04-20T10:15:00+00:00,,,,None,None,None,None
1491,2024-04-20DL9617None,2024-04-20,landed,9617,DL9617,DAL9617,Delta Air Lines,DL,DAL,Schiphol,...,,2024-04-20T11:10:00+00:00,2024-04-20T11:10:00+00:00,2024-04-20T11:09:00+00:00,2024-04-20T11:09:00+00:00,2024-04-20T11:09:00+00:00,None,None,None,None


## Receiveing and storing data

Brief description about the scripts created to retrieves data from the API and store into SQL database.

### `source_to_landing`
This script was created to store the function responsible for getting the API's data, given a data range and an airline.
The function is called `retrieve_hist_flights(start_date:str, end_date:str, airline:str, option:str, api_key:str)`, which has the description documented inside the code:

`This function retrieves the data from historical flights of a specific date

    Args:
        start_date(str): Start date from the range. Format:"%Y-%m-%d"
        end_date(str): End date from the range. Format:"%Y-%m-%d"
        airline(str): Airline name, IATA code or ICAO code. According to the option explicited on the args.
        option(str): "name", "iata" or "icao". Option for the airline filter.
        api_key(str): Your API KEY.
    Returns:
        list: List containing all Csv files paths.`
        

### `landing_to_sql`
This script was created to store the function responsible for uploading the data received from the function above, given .csv file path.
The function is called `upload_csv(csv_path:str)`, which has the description documented inside the code:

`This function creates the database (if not exists), and populate it with a csv file retieved from the API.

    Args:
        csv_path (str): Csv file path. Ex: "csv-files/2024-04-15_dl.csv".`

### Examples using both functions:

*OBS.: If you are not going to retrieve any new data at this moment, you can skip it and work with the pre-loaded DB*

In [6]:
csv_paths = source_to_landing.retrieve_hist_flights("2024-04-16","2024-04-20","dl","iata","YOUR API KEY")

Getting data from 2024-04-17
Taking data from offset:0
Taking data from offset:100
Taking data from offset:200
Taking data from offset:300
Taking data from offset:400
Taking data from offset:500
Taking data from offset:600
Taking data from offset:700
Taking data from offset:800
Taking data from offset:900
Taking data from offset:1000
Taking data from offset:1100
Taking data from offset:1200
Taking data from offset:1300
Taking data from offset:1400
Taking data from offset:1500
Taking data from offset:1600
Taking data from offset:1700
Taking data from offset:1800
Taking data from offset:1900
Taking data from offset:2000
Taking data from offset:2100
Taking data from offset:2200
Taking data from offset:2300
Taking data from offset:2400
Taking data from offset:2500
Taking data from offset:2600
Taking data from offset:2700
Taking data from offset:2800
Taking data from offset:2900
Taking data from offset:3000
Taking data from offset:3100
Taking data from offset:3200
Taking data from offset:33

In [9]:
for paths in csv_paths:
    landing_to_sql.upload_csv(paths)

Successfully populated db with data from : csv-files/2024-04-16_dl.csv
Successfully populated db with data from : csv-files/2024-04-17_dl.csv
Successfully populated db with data from : csv-files/2024-04-18_dl.csv
Successfully populated db with data from : csv-files/2024-04-19_dl.csv
Successfully populated db with data from : csv-files/2024-04-20_dl.csv


# Data Analysis
**IMPORTANT:All these analysis, that are already printed, are based on the 5 days period of one airline. For further analysis, querys modifications need to be done**

### Total number of flights during the range date

In [73]:
temp_df = query_db("SELECT COUNT(*) as count FROM flights")
                         
total_flights = temp_df["count"][0]

print(f"{total_flights} flights")

33059 flights


### Average flight duration

In [78]:
temp_df = query_db("""SELECT AVG(julianday(TIME(arrival_actual)) - julianday(TIME(depart_actual)))*1440 as avg FROM flights
                          WHERE arrival_actual IS NOT False
                          """)

avg_flight_dur = temp_df["avg"][0]

print(f"{avg_flight_dur.round(1)} minutes")

56.9 minutes


### Average flights per day

In [83]:
temp_df = query_db("""SELECT AVG(count) as avg FROM (SELECT COUNT(*) as count FROM flights GROUP BY flight_date)""")

flights_p_day = temp_df["avg"][0]

print(f"{flights_p_day} flights per day")

6611.8 flights per day


### Average flights arrival per day per airport

In [96]:
temp_df = query_db("""SELECT AVG(count) as avg FROM 
                        (
                        SELECT arrival_iata,flight_date, COUNT(*) as count FROM flights GROUP BY arrival_iata,flight_date
                        )""")

flights_p_day = temp_df["avg"][0]

print(f"{flights_p_day.round(0)} flights arrivals per day")

12.0 flights arrivals per day


### Average delay (departure)

In [99]:
temp_df = query_db("""SELECT AVG(depart_delay) as avg FROM flights""")

delay_p_day = temp_df["avg"][0]

print(f"{delay_p_day.round(0)} minutes, in departure, per day")

25.0 minutes, in departure, per day


### Average delay (arrival)

In [100]:
temp_df = query_db("""SELECT AVG(arrival_delay) as avg FROM flights""")

delay_p_day = temp_df["avg"][0]

print(f"{delay_p_day.round(0)} minutes, in arrival, per day")

6.0 minutes, in arrival, per day


### Flights per day

In [108]:
temp_df = query_db("""SELECT COUNT(*) as Flights, flight_date as date FROM flights  GROUP BY flight_date""")

fig = px.bar(temp_df, x="date", y="Flights")
fig.show()

### TOP 10 Airports with more flights in departure

In [119]:
temp_df = query_db("""SELECT COUNT(*) as Flights, depart_airport as airport 
                        FROM flights  
                        GROUP BY depart_airport 
                        ORDER BY Flights DESC
                        LIMIT 10""")


fig = px.bar(temp_df, x="airport", y="Flights")
fig.show()

### TOP 10 Airports with more flights in arrival

In [123]:
temp_df = query_db("""SELECT COUNT(*) as Flights, arrival_airport as airport 
                        FROM flights  
                        GROUP BY arrival_airport 
                        ORDER BY Flights DESC
                        LIMIT 10""")

fig = px.bar(temp_df, x="airport", y="Flights")
fig.show()

### Flights though the dates

In [158]:
temp_df = query_db("""SELECT COUNT(*) as Flights, flight_date as date
                        FROM flights  
                        GROUP BY flight_date 
                        """)

fig = px.line(temp_df, x="date", y="Flights")
fig.show()

### Total aircrafts labeled according to number of flights

In [156]:
temp_df = query_db("""WITH temp_table AS (SELECT COUNT(*) as Flights, aircraft_registration as aircraft 
                        FROM flights 
                        WHERE aircraft_registration != 'None'
                        GROUP BY aircraft_registration 
                        ORDER BY Flights DESC),
                    temp_table2 as (SELECT *, 
                    CASE WHEN Flights <20 AND Flights >=15 THEN "15 or more"
                    WHEN Flights <15 AND Flights >=10 THEN "14 - 10"
                    WHEN Flights <10 AND Flights >=5 THEN "9 - 5"
                    ELSE "5 or less"
                    END AS Number_of_Flights
                    FROM temp_table)
                    SELECT COUNT(aircraft) as count ,Number_of_Flights FROM temp_table2 GROUP BY Number_of_Flights
                        """)
fig = px.pie(temp_df, names="Number_of_Flights",  values="count")
fig.show()